In [1]:
library(lme4)
library(blme)
library(ggplot2)
library(stringr)
library(psycho)
library(glmnet)
library(car)


Loading required package: Matrix


Attaching package: ‘psycho’


The following object is masked from ‘package:lme4’:

    golden


Loaded glmnet 4.1-7

Loading required package: carData



In [2]:
# Set ggplot theme
base_size <- 20
theme_set(theme_bw(base_size=base_size) +
            theme(#panel.grid.major=element_blank(),
              panel.grid.minor=element_blank(),
              axis.title.y=element_text(angle=90,vjust=0.5),
              axis.text.x=element_text(angle=0, hjust=0.5),
              axis.title.x=element_blank()))

In [4]:
correct_vs_length_and_prob <- function(df, include_output_chars=TRUE, include_input_chars=TRUE, include_intercept=TRUE){
    if (include_output_chars & include_input_chars & include_intercept){
        model <- glmer(correct ~ (1|index) + input_logprob + output_logprob + input_ntokens + output_ntokens + input_nchars + output_nchars, 
               data=df, family=binomial)
    } else if (include_output_chars & !include_input_chars & include_intercept){
        model <- glmer(correct ~ (1|index) + input_logprob + output_logprob + input_ntokens + output_ntokens + output_nchars, 
               data=df, family=binomial)
    } else if (!include_output_chars & include_input_chars & include_intercept){
        model <- glmer(correct ~ (1|index) + input_logprob + output_logprob + input_ntokens + output_ntokens + input_nchars, 
               data=df, family=binomial)

    
    } else if (include_output_chars & include_input_chars & !include_intercept){
        model <- glm(correct ~ input_logprob + output_logprob + input_ntokens + output_ntokens + input_nchars + output_nchars, 
               data=df, family=binomial)
    } else if (include_output_chars & !include_input_chars & !include_intercept){
        model <- glm(correct ~ input_logprob + output_logprob + input_ntokens + output_ntokens + output_nchars, 
               data=df, family=binomial)
    } else if (!include_output_chars & include_input_chars & !include_intercept){
        model <- glm(correct ~ input_logprob + output_logprob + input_ntokens + output_ntokens + input_nchars, 
               data=df, family=binomial)
    }
  
    
  return(model)
}

In [5]:
scale_df <- function(df) {
    new_df <- data.frame(scale(df[2:7]))
    new_df$index <- as.factor(df$index)
    new_df$correct <- df$correct
    
    return(new_df)
}

In [6]:
predictor_cors <- function(df) {
    predictors <- c("input_logprob", "output_logprob", "input_nchars", "input_ntokens", "output_nchars", "output_ntokens")
    
    for (pred1 in predictors) {
        for (pred2 in predictors) {
            cat(pred1, pred2, cor(df[pred1], df[pred2]), "\n")
        }
        cat("\n")
    }
    
}

# Rot-13

In [7]:
# Read in data
rot13enc_gpt4_df <- read.table(file = 'table_rot13enc_gpt-4.tsv', sep = '\t', header = TRUE)
rot13enc_gpt35_df <- read.table(file = 'table_rot13enc_gpt-3.5-turbo.tsv', sep = '\t', header = TRUE)

rot13dec_gpt4_df <- read.table(file = 'table_rot13dec_gpt-4.tsv', sep = '\t', header = TRUE)
rot13dec_gpt35_df <- read.table(file = 'table_rot13dec_gpt-3.5-turbo.tsv', sep = '\t', header = TRUE)

In [8]:
# Z-score data
scaled_rot13enc_gpt4_df <- scale_df(rot13enc_gpt4_df)
scaled_rot13enc_gpt35_df <- scale_df(rot13enc_gpt35_df)

scaled_rot13dec_gpt4_df <- scale_df(rot13dec_gpt4_df)
scaled_rot13dec_gpt35_df <- scale_df(rot13dec_gpt35_df)


In [9]:
# Print correlations among predictors
predictor_cors(scaled_rot13enc_gpt4_df)

input_logprob input_logprob 1 
input_logprob output_logprob 0.671355 
input_logprob input_nchars -0.6635202 
input_logprob input_ntokens -0.6448557 
input_logprob output_nchars -0.6635202 
input_logprob output_ntokens -0.6639417 

output_logprob input_logprob 0.671355 
output_logprob output_logprob 1 
output_logprob input_nchars -0.9936365 
output_logprob input_ntokens -0.941151 
output_logprob output_nchars -0.9936365 
output_logprob output_ntokens -0.9914956 

input_nchars input_logprob -0.6635202 
input_nchars output_logprob -0.9936365 
input_nchars input_nchars 1 
input_nchars input_ntokens 0.9581319 
input_nchars output_nchars 1 
input_nchars output_ntokens 0.9946449 

input_ntokens input_logprob -0.6448557 
input_ntokens output_logprob -0.941151 
input_ntokens input_nchars 0.9581319 
input_ntokens input_ntokens 1 
input_ntokens output_nchars 0.9581319 
input_ntokens output_ntokens 0.9517609 

output_nchars input_logprob -0.6635202 
output_nchars output_logprob -0.9936365 
output_

In [10]:
predictor_cors(scaled_rot13enc_gpt35_df)

input_logprob input_logprob 1 
input_logprob output_logprob 0.671355 
input_logprob input_nchars -0.6635202 
input_logprob input_ntokens -0.6448557 
input_logprob output_nchars -0.6635202 
input_logprob output_ntokens -0.6639417 

output_logprob input_logprob 0.671355 
output_logprob output_logprob 1 
output_logprob input_nchars -0.9936365 
output_logprob input_ntokens -0.941151 
output_logprob output_nchars -0.9936365 
output_logprob output_ntokens -0.9914956 

input_nchars input_logprob -0.6635202 
input_nchars output_logprob -0.9936365 
input_nchars input_nchars 1 
input_nchars input_ntokens 0.9581319 
input_nchars output_nchars 1 
input_nchars output_ntokens 0.9946449 

input_ntokens input_logprob -0.6448557 
input_ntokens output_logprob -0.941151 
input_ntokens input_nchars 0.9581319 
input_ntokens input_ntokens 1 
input_ntokens output_nchars 0.9581319 
input_ntokens output_ntokens 0.9517609 

output_nchars input_logprob -0.6635202 
output_nchars output_logprob -0.9936365 
output_

In [11]:
predictor_cors(scaled_rot13dec_gpt4_df)

input_logprob input_logprob 1 
input_logprob output_logprob 0.671355 
input_logprob input_nchars -0.9936365 
input_logprob input_ntokens -0.9914956 
input_logprob output_nchars -0.9936365 
input_logprob output_ntokens -0.941151 

output_logprob input_logprob 0.671355 
output_logprob output_logprob 1 
output_logprob input_nchars -0.6635202 
output_logprob input_ntokens -0.6639417 
output_logprob output_nchars -0.6635202 
output_logprob output_ntokens -0.6448557 

input_nchars input_logprob -0.9936365 
input_nchars output_logprob -0.6635202 
input_nchars input_nchars 1 
input_nchars input_ntokens 0.9946449 
input_nchars output_nchars 1 
input_nchars output_ntokens 0.9581319 

input_ntokens input_logprob -0.9914956 
input_ntokens output_logprob -0.6639417 
input_ntokens input_nchars 0.9946449 
input_ntokens input_ntokens 1 
input_ntokens output_nchars 0.9946449 
input_ntokens output_ntokens 0.9517609 

output_nchars input_logprob -0.9936365 
output_nchars output_logprob -0.6635202 
output

In [12]:
predictor_cors(scaled_rot13dec_gpt35_df)

input_logprob input_logprob 1 
input_logprob output_logprob 0.671355 
input_logprob input_nchars -0.9936365 
input_logprob input_ntokens -0.9914956 
input_logprob output_nchars -0.9936365 
input_logprob output_ntokens -0.941151 

output_logprob input_logprob 0.671355 
output_logprob output_logprob 1 
output_logprob input_nchars -0.6635202 
output_logprob input_ntokens -0.6639417 
output_logprob output_nchars -0.6635202 
output_logprob output_ntokens -0.6448557 

input_nchars input_logprob -0.9936365 
input_nchars output_logprob -0.6635202 
input_nchars input_nchars 1 
input_nchars input_ntokens 0.9946449 
input_nchars output_nchars 1 
input_nchars output_ntokens 0.9581319 

input_ntokens input_logprob -0.9914956 
input_ntokens output_logprob -0.6639417 
input_ntokens input_nchars 0.9946449 
input_ntokens input_ntokens 1 
input_ntokens output_nchars 0.9946449 
input_ntokens output_ntokens 0.9517609 

output_nchars input_logprob -0.9936365 
output_nchars output_logprob -0.6635202 
output

In [13]:
intercepts <- FALSE
rot13enc_gpt4_model <- correct_vs_length_and_prob(scaled_rot13enc_gpt4_df, include_output_chars=FALSE, include_intercept=intercepts)
rot13enc_gpt35_model <- correct_vs_length_and_prob(scaled_rot13enc_gpt35_df, include_output_chars=FALSE, include_intercept=FALSE)

rot13dec_gpt4_model <- correct_vs_length_and_prob(scaled_rot13dec_gpt4_df, include_output_chars=FALSE, include_intercept=intercepts)
rot13dec_gpt35_model <- correct_vs_length_and_prob(scaled_rot13dec_gpt35_df, include_output_chars=FALSE, include_intercept=intercepts)

Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”


In [14]:
summary(rot13enc_gpt4_model)


Call:
glm(formula = correct ~ input_logprob + output_logprob + input_ntokens + 
    output_ntokens + input_nchars, family = binomial, data = df)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-2.06663  -0.44366  -0.14606  -0.01354   2.97169  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)     -4.1193     0.5713  -7.211 5.57e-13 ***
input_logprob    1.2158     0.4594   2.646  0.00814 ** 
output_logprob   7.0316     3.1573   2.227  0.02594 *  
input_ntokens    2.9504     0.9824   3.003  0.00267 ** 
output_ntokens  -1.5485     2.5145  -0.616  0.53802    
input_nchars     3.0854     3.3378   0.924  0.35529    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 246.58  on 299  degrees of freedom
Residual deviance: 156.14  on 294  degrees of freedom
AIC: 168.14

Number of Fisher Scoring iterations: 7


In [15]:
summary(rot13enc_gpt35_model)


Call:
glm(formula = correct ~ input_logprob + output_logprob + input_ntokens + 
    output_ntokens + input_nchars, family = binomial, data = df)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-1.28769  -0.08616  -0.01909  -0.00030   3.10549  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)    -10.7542     2.5495  -4.218 2.46e-05 ***
input_logprob   -0.2191     1.0850  -0.202   0.8399    
output_logprob  19.6737     9.8166   2.004   0.0451 *  
input_ntokens    4.3058     2.6034   1.654   0.0982 .  
output_ntokens   4.5089     7.3753   0.611   0.5410    
input_nchars     3.5266     8.5951   0.410   0.6816    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 80.845  on 299  degrees of freedom
Residual deviance: 37.569  on 294  degrees of freedom
AIC: 49.569

Number of Fisher Scoring iterations: 10


In [16]:
summary(rot13dec_gpt4_model)


Call:
glm(formula = correct ~ input_logprob + output_logprob + input_ntokens + 
    output_ntokens + input_nchars, family = binomial, data = df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.4640  -0.8038  -0.4112   0.8902   2.5726  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)     -1.4509     0.1977  -7.340 2.13e-13 ***
input_logprob    1.4829     1.7607   0.842   0.3996    
output_logprob   1.7068     0.3173   5.380 7.46e-08 ***
input_ntokens   -3.0153     1.6183  -1.863   0.0624 .  
output_ntokens   0.5686     0.6128   0.928   0.3535    
input_nchars     4.0308     2.1224   1.899   0.0575 .  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 357.64  on 299  degrees of freedom
Residual deviance: 285.30  on 294  degrees of freedom
AIC: 297.3

Number of Fisher Scoring iterations: 6


In [17]:
summary(rot13dec_gpt35_model)


Call:
glm(formula = correct ~ input_logprob + output_logprob + input_ntokens + 
    output_ntokens + input_nchars, family = binomial, data = df)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-1.44662  -0.32654  -0.10349  -0.01189   2.92757  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)     -5.0436     0.7792  -6.473 9.62e-11 ***
input_logprob    3.2597     3.3367   0.977   0.3286    
output_logprob   3.8136     0.8978   4.247 2.16e-05 ***
input_ntokens   -5.2379     2.7949  -1.874   0.0609 .  
output_ntokens   2.4683     1.0515   2.347   0.0189 *  
input_nchars     5.5853     3.6081   1.548   0.1216    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 203.69  on 299  degrees of freedom
Residual deviance: 125.76  on 294  degrees of freedom
AIC: 137.76

Number of Fisher Scoring iterations: 8


In [18]:
vif(rot13enc_gpt4_model)

input_logprob output_logprob  input_ntokens output_ntokens   input_nchars 
      1.130600      39.299988       8.654038      29.877453      52.790918

In [19]:
vif(rot13enc_gpt35_model)

input_logprob output_logprob  input_ntokens output_ntokens   input_nchars 
      1.256390      43.209014       7.383102      30.032245      48.272562

In [20]:
vif(rot13dec_gpt4_model)

input_logprob output_logprob  input_ntokens output_ntokens   input_nchars 
     97.900802       1.389986      83.241865      13.595196     146.200936

In [21]:
vif(rot13dec_gpt35_model)

input_logprob output_logprob  input_ntokens output_ntokens   input_nchars 
     57.759362       1.365018      42.021312       9.277213      74.992264

# Pig Latin

In [26]:
# Read in data
pigenc_gpt4_df <- read.table(file = 'table_pig_ay_enc_gpt-4.tsv', sep = '\t', header = TRUE)
pigenc_gpt35_df <- read.table(file = 'table_pig_ay_enc_gpt-3.5-turbo.tsv', sep = '\t', header = TRUE)

pigdec_gpt4_df <- read.table(file = 'table_pig_ay_dec_gpt-4.tsv', sep = '\t', header = TRUE)
pigdec_gpt35_df <- read.table(file = 'table_pig_ay_dec_gpt-3.5-turbo.tsv', sep = '\t', header = TRUE)

In [27]:
# Z-score data
scaled_pigenc_gpt4_df <- scale_df(pigenc_gpt4_df)
scaled_pigenc_gpt35_df <- scale_df(pigenc_gpt35_df)

scaled_pigdec_gpt4_df <- scale_df(pigdec_gpt4_df)
scaled_pigdec_gpt35_df <- scale_df(pigdec_gpt35_df)

In [28]:
# Print correlations among predictors
predictor_cors(scaled_pigenc_gpt4_df)

input_logprob input_logprob 1 
input_logprob output_logprob 0.697457 
input_logprob input_nchars -0.6764854 
input_logprob input_ntokens -0.6690056 
input_logprob output_nchars -0.6771126 
input_logprob output_ntokens -0.6722721 

output_logprob input_logprob 0.697457 
output_logprob output_logprob 1 
output_logprob input_nchars -0.9657046 
output_logprob input_ntokens -0.957093 
output_logprob output_nchars -0.970346 
output_logprob output_ntokens -0.9748658 

input_nchars input_logprob -0.6764854 
input_nchars output_logprob -0.9657046 
input_nchars input_nchars 1 
input_nchars input_ntokens 0.9568471 
input_nchars output_nchars 0.9975957 
input_nchars output_ntokens 0.9664903 

input_ntokens input_logprob -0.6690056 
input_ntokens output_logprob -0.957093 
input_ntokens input_nchars 0.9568471 
input_ntokens input_ntokens 1 
input_ntokens output_nchars 0.9708763 
input_ntokens output_ntokens 0.9796248 

output_nchars input_logprob -0.6771126 
output_nchars output_logprob -0.970346 
o

In [29]:
predictor_cors(scaled_pigenc_gpt35_df)

input_logprob input_logprob 1 
input_logprob output_logprob 0.697457 
input_logprob input_nchars -0.6764854 
input_logprob input_ntokens -0.6690056 
input_logprob output_nchars -0.6771126 
input_logprob output_ntokens -0.6722721 

output_logprob input_logprob 0.697457 
output_logprob output_logprob 1 
output_logprob input_nchars -0.9657046 
output_logprob input_ntokens -0.957093 
output_logprob output_nchars -0.970346 
output_logprob output_ntokens -0.9748658 

input_nchars input_logprob -0.6764854 
input_nchars output_logprob -0.9657046 
input_nchars input_nchars 1 
input_nchars input_ntokens 0.9568471 
input_nchars output_nchars 0.9975957 
input_nchars output_ntokens 0.9664903 

input_ntokens input_logprob -0.6690056 
input_ntokens output_logprob -0.957093 
input_ntokens input_nchars 0.9568471 
input_ntokens input_ntokens 1 
input_ntokens output_nchars 0.9708763 
input_ntokens output_ntokens 0.9796248 

output_nchars input_logprob -0.6771126 
output_nchars output_logprob -0.970346 
o

In [30]:
predictor_cors(scaled_pigdec_gpt4_df)

input_logprob input_logprob 1 
input_logprob output_logprob 0.697457 
input_logprob input_nchars -0.970346 
input_logprob input_ntokens -0.9748658 
input_logprob output_nchars -0.9657046 
input_logprob output_ntokens -0.957093 

output_logprob input_logprob 0.697457 
output_logprob output_logprob 1 
output_logprob input_nchars -0.6771126 
output_logprob input_ntokens -0.6722721 
output_logprob output_nchars -0.6764854 
output_logprob output_ntokens -0.6690056 

input_nchars input_logprob -0.970346 
input_nchars output_logprob -0.6771126 
input_nchars input_nchars 1 
input_nchars input_ntokens 0.9779298 
input_nchars output_nchars 0.9975957 
input_nchars output_ntokens 0.9708763 

input_ntokens input_logprob -0.9748658 
input_ntokens output_logprob -0.6722721 
input_ntokens input_nchars 0.9779298 
input_ntokens input_ntokens 1 
input_ntokens output_nchars 0.9664903 
input_ntokens output_ntokens 0.9796248 

output_nchars input_logprob -0.9657046 
output_nchars output_logprob -0.6764854 


In [31]:
predictor_cors(scaled_pigdec_gpt35_df)

input_logprob input_logprob 1 
input_logprob output_logprob 0.697457 
input_logprob input_nchars -0.970346 
input_logprob input_ntokens -0.9748658 
input_logprob output_nchars -0.9657046 
input_logprob output_ntokens -0.957093 

output_logprob input_logprob 0.697457 
output_logprob output_logprob 1 
output_logprob input_nchars -0.6771126 
output_logprob input_ntokens -0.6722721 
output_logprob output_nchars -0.6764854 
output_logprob output_ntokens -0.6690056 

input_nchars input_logprob -0.970346 
input_nchars output_logprob -0.6771126 
input_nchars input_nchars 1 
input_nchars input_ntokens 0.9779298 
input_nchars output_nchars 0.9975957 
input_nchars output_ntokens 0.9708763 

input_ntokens input_logprob -0.9748658 
input_ntokens output_logprob -0.6722721 
input_ntokens input_nchars 0.9779298 
input_ntokens input_ntokens 1 
input_ntokens output_nchars 0.9664903 
input_ntokens output_ntokens 0.9796248 

output_nchars input_logprob -0.9657046 
output_nchars output_logprob -0.6764854 


In [55]:
intercepts <- TRUE
pigenc_gpt4_model <- correct_vs_length_and_prob(scaled_pigenc_gpt4_df, include_output_chars=TRUE, include_intercept=intercepts)
pigenc_gpt35_model <- correct_vs_length_and_prob(scaled_pigenc_gpt35_df, include_output_chars=TRUE, include_intercept=intercepts)

pigdec_gpt4_model <- correct_vs_length_and_prob(scaled_pigdec_gpt4_df, include_output_chars=TRUE, include_intercept=FALSE)
pigdec_gpt35_model <- correct_vs_length_and_prob(scaled_pigdec_gpt35_df, include_output_chars=TRUE, include_intercept=intercepts)

In [56]:
summary(pigenc_gpt4_model) 

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
correct ~ (1 | index) + input_logprob + output_logprob + input_ntokens +  
    output_ntokens + input_nchars + output_nchars
   Data: df

     AIC      BIC   logLik deviance df.resid 
   353.4    383.1   -168.7    337.4      292 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.4042 -0.4186 -0.2318  0.4768  1.8299 

Random effects:
 Groups Name        Variance Std.Dev.
 index  (Intercept) 4.098    2.024   
Number of obs: 300, groups:  index, 100

Fixed effects:
                Estimate Std. Error z value Pr(>|z|)  
(Intercept)     -0.67742    0.28409  -2.385   0.0171 *
input_logprob   -0.03599    0.24006  -0.150   0.8808  
output_logprob   2.95441    1.26321   2.339   0.0193 *
input_ntokens    3.27961    1.65590   1.981   0.0476 *
output_ntokens   1.81639    1.75227   1.037   0.2999  
input_nchars    10.59354    6.16256   1.719   0.0856 .
o

In [57]:
summary(pigenc_gpt35_model)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
correct ~ (1 | index) + input_logprob + output_logprob + input_ntokens +  
    output_ntokens + input_nchars + output_nchars
   Data: df

     AIC      BIC   logLik deviance df.resid 
   314.1    343.8   -149.1    298.1      292 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.2106 -0.3772 -0.2486  0.5059  2.3940 

Random effects:
 Groups Name        Variance Std.Dev.
 index  (Intercept) 3.304    1.818   
Number of obs: 300, groups:  index, 100

Fixed effects:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)     -1.8830     0.3586  -5.250 1.52e-07 ***
input_logprob    0.2070     0.2587   0.800    0.424    
output_logprob   1.9055     1.2514   1.523    0.128    
input_ntokens    2.4930     1.6237   1.535    0.125    
output_ntokens  -1.9333     1.8975  -1.019    0.308    
input_nchars    -4.8041     6.0234  -0.798    0.42

In [58]:
summary(pigdec_gpt4_model)


Call:
glm(formula = correct ~ input_logprob + output_logprob + input_ntokens + 
    output_ntokens + input_nchars + output_nchars, family = binomial, 
    data = df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.7284  -0.8505   0.4236   0.7521   2.4140  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)      0.5198     0.1491   3.487 0.000488 ***
input_logprob    1.4257     0.7709   1.849 0.064402 .  
output_logprob   1.9253     0.2431   7.918  2.4e-15 ***
input_ntokens    1.3736     1.1236   1.223 0.221505    
output_ntokens  -1.1396     0.9170  -1.243 0.213953    
input_nchars     1.4535     4.2228   0.344 0.730698    
output_nchars    1.1972     3.4386   0.348 0.727713    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 406.12  on 299  degrees of freedom
Residual deviance: 298.86  on 293  degrees of freedom
AIC: 312.86

Number of

In [59]:
summary(pigdec_gpt35_model)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
correct ~ (1 | index) + input_logprob + output_logprob + input_ntokens +  
    output_ntokens + input_nchars + output_nchars
   Data: df

     AIC      BIC   logLik deviance df.resid 
   235.7    265.3   -109.8    219.7      292 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.3494 -0.2424 -0.0334  0.1206  6.2121 

Random effects:
 Groups Name        Variance Std.Dev.
 index  (Intercept) 5.68     2.383   
Number of obs: 300, groups:  index, 100

Fixed effects:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)     -3.3753     0.9011  -3.746  0.00018 ***
input_logprob    2.4499     1.8008   1.360  0.17370    
output_logprob   5.2103     1.3345   3.904 9.45e-05 ***
input_ntokens   -0.5393     2.3738  -0.227  0.82026    
output_ntokens  -3.8063     2.4813  -1.534  0.12503    
input_nchars     8.2307     9.8044   0.839  0.4012

In [60]:
vif(pigenc_gpt4_model)

input_logprob output_logprob  input_ntokens output_ntokens   input_nchars 
      1.316986      17.871016      32.851971      35.952299     445.372168 
 output_nchars 
    673.529909

In [61]:
vif(pigenc_gpt35_model)

input_logprob output_logprob  input_ntokens output_ntokens   input_nchars 
      1.292814      17.876312      34.399131      43.320710     441.346083 
 output_nchars 
    672.539492

In [62]:
vif(pigdec_gpt4_model)

input_logprob output_logprob  input_ntokens output_ntokens   input_nchars 
     26.135763       2.430346      52.497049      34.483101     733.105539 
 output_nchars 
    489.195865

In [63]:
vif(pigdec_gpt35_model)

input_logprob output_logprob  input_ntokens output_ntokens   input_nchars 
     25.788197       4.093621      46.375165      52.407965     822.271716 
 output_nchars 
    544.860529